In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
# Load the Dataset
data = pd.read_csv('The-Complete-Neural-Networks-Bootcamp-Theory-Applications-master/diabetes.csv')

In [3]:
data

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,negative
764,2,122,70,27,0,36.8,27,negative
765,5,121,72,23,112,26.2,30,negative
766,1,126,60,0,0,30.1,47,positive


In [4]:
# X and y 
X = data.iloc[:,:-1]
y_str = list(data.iloc[:,-1])

In [44]:
X = np.array(X, dtype='float32')

In [45]:
y = []
for i in y_str:
    if i == 'positive':
        y.append(1)
    elif i == 'negative':
        y.append(0)

In [46]:
# Convert list to array
y = np.array(y, dtype='float32')

In [47]:
# Data Normalization: All the feature should have the range between velues (-1, 1). Standardization technique
sc = StandardScaler()
X = sc.fit_transform(X)

In [48]:
X

array([[ 0.63994724,  0.8483238 ,  0.14964075, ..., -0.6928906 ,
         0.20401277,  1.4259953 ],
       [-0.84488505, -1.1233964 , -0.16054575, ..., -0.6928906 ,
        -0.68442196, -0.1906719 ],
       [ 1.2338802 ,  1.9437239 , -0.26394126, ..., -0.6928906 ,
        -1.1032555 , -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ...,  0.27959377,
        -0.7351896 , -0.27575967],
       [-0.84488505,  0.1597866 , -0.47073224, ..., -0.6928906 ,
        -0.24020459,  1.1707321 ],
       [-0.84488505, -0.8730192 ,  0.04624526, ..., -0.6928906 ,
        -0.20212881, -0.87137395]], dtype=float32)

In [49]:
# Convert to pytorch tensors
X = torch.tensor(X)
y = torch.tensor(y)

In [50]:
X.shape, y.shape

(torch.Size([768, 7]), torch.Size([768]))

In [51]:
y.view(-1,1).shape

torch.Size([768, 1])

In [52]:
y = y.unsqueeze(1)

In [53]:
y.shape, X.shape

(torch.Size([768, 1]), torch.Size([768, 7]))

In [54]:
# Build custom Dataset
class Dataset(Dataset):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return len(self.x)

In [55]:
dataset = Dataset(X,y)

In [56]:
len(dataset)

768

In [57]:
# Load data from dataloader
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset = dataset, 
           batch_size = 32, shuffle = True)

In [58]:
train_loader

In [63]:
print(f"There are {len(train_loader)} batches in the dataset")
for x,y in train_loader:
    print("For one iteration(batch), there is:")
    print(f"Data:  {x.shape}")
    print(f"Labels:  {y.shape}")
    print(f"Data:  {x.dtype}")
    print(f"Labels:  {y.dtype}")
    break

There are 24 batches in the dataset
For one iteration(batch), there is:
Data:  torch.Size([32, 7])
Labels:  torch.Size([32, 1])
Data:  torch.float32
Labels:  torch.float32


In [67]:
# Build the model
class Model(nn.Module):
    
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [68]:
# Objectof the model
net = Model(7, 1)
criterion = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [70]:
# Train the neural network
epochs = 200
for epoch in range(epochs):
    for inputs, labels in train_loader:
        # Forward Prop
        outputs = net(inputs)
        # Loss Calculation
        loss = criterion(outputs, labels)
        
        # Backpropagation
        # 1. Clear the Gradient Buffer
        optimizer.zero_grad()
        # 2. Calculate the gradients 
        loss.backward()
        # 3. Update the weights
        optimizer.step()
        
    # Accuracy Calculation
    output = (outputs>0.5).float()
    accuracy = (output == labels).float().mean()
    # Print Statistics 
    print(" Epoch: {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,epochs,loss,accuracy))

 Epoch: 1/200, Loss: 0.488, Accuracy: 0.750
 Epoch: 2/200, Loss: 0.554, Accuracy: 0.656
 Epoch: 3/200, Loss: 0.433, Accuracy: 0.781
 Epoch: 4/200, Loss: 0.380, Accuracy: 0.781
 Epoch: 5/200, Loss: 0.557, Accuracy: 0.656
 Epoch: 6/200, Loss: 0.526, Accuracy: 0.750
 Epoch: 7/200, Loss: 0.579, Accuracy: 0.719
 Epoch: 8/200, Loss: 0.403, Accuracy: 0.781
 Epoch: 9/200, Loss: 0.414, Accuracy: 0.719
 Epoch: 10/200, Loss: 0.385, Accuracy: 0.812
 Epoch: 11/200, Loss: 0.417, Accuracy: 0.812
 Epoch: 12/200, Loss: 0.414, Accuracy: 0.750
 Epoch: 13/200, Loss: 0.659, Accuracy: 0.625
 Epoch: 14/200, Loss: 0.483, Accuracy: 0.719
 Epoch: 15/200, Loss: 0.535, Accuracy: 0.750
 Epoch: 16/200, Loss: 0.481, Accuracy: 0.781
 Epoch: 17/200, Loss: 0.491, Accuracy: 0.719
 Epoch: 18/200, Loss: 0.533, Accuracy: 0.844
 Epoch: 19/200, Loss: 0.363, Accuracy: 0.781
 Epoch: 20/200, Loss: 0.363, Accuracy: 0.781
 Epoch: 21/200, Loss: 0.381, Accuracy: 0.812
 Epoch: 22/200, Loss: 0.596, Accuracy: 0.562
 Epoch: 23/200, Los

 Epoch: 186/200, Loss: 0.346, Accuracy: 0.812
 Epoch: 187/200, Loss: 0.386, Accuracy: 0.844
 Epoch: 188/200, Loss: 0.353, Accuracy: 0.812
 Epoch: 189/200, Loss: 0.443, Accuracy: 0.781
 Epoch: 190/200, Loss: 0.436, Accuracy: 0.812
 Epoch: 191/200, Loss: 0.688, Accuracy: 0.594
 Epoch: 192/200, Loss: 0.224, Accuracy: 0.906
 Epoch: 193/200, Loss: 0.387, Accuracy: 0.844
 Epoch: 194/200, Loss: 0.433, Accuracy: 0.750
 Epoch: 195/200, Loss: 0.501, Accuracy: 0.844
 Epoch: 196/200, Loss: 0.419, Accuracy: 0.719
 Epoch: 197/200, Loss: 0.412, Accuracy: 0.750
 Epoch: 198/200, Loss: 0.438, Accuracy: 0.812
 Epoch: 199/200, Loss: 0.214, Accuracy: 0.938
 Epoch: 200/200, Loss: 0.478, Accuracy: 0.781
